### Infix to Prefix

Consider the following grammar for arithmetic expressions. It has attribute rules for `integer` and `digit`; the attributes are not used:

    expression → ws term { '+' ws term }
    term → factor { '*' ws factor }
    factor → (integer | '(' expression ')' ) ws
    integer(p) → digit(p) { digit(q) « p := q + p» } ws
    digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    ws → { ' ' }

Here is a recursive descent parser for this grammar:

In [ ]:
pos: int; sym: str; src: str

def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos + 1
    else: sym = chr(0) # end of input symbol

def expression() -> str:
    # expression → term { '+' ws term }
    ws(); term()
    while sym == '+': nxt(); ws(); term() 

def term() -> str:
    # term → factor { '*' ws factor }
    p = factor()
    while sym == '*': nxt(); ws(); factor()

def factor() -> str:
    # factor → (integer | '(' expression ')' ) ws
    if '0' <= sym <= '9': integer()
    elif sym == '(':
        nxt(); expression()
        if sym == ')': nxt(); ws()
        else: raise Exception("')' expected at " + str(pos))
    else: raise Exception("invalid character at " + str(pos))

def integer() -> str:
    # integer(p) → digit(p) { digit(q) « p := q + p» } ws
    p = digit()
    while '0' <= sym <= '9': p += digit()
    ws()
    return p

def digit() -> str:
    # digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    # '0' <= sym <= '9'
    p = sym; nxt()
    return p

def ws():
    # ws → { ' ' }
    while sym == ' ': nxt()

def convert(s) -> str:
    global src, pos;
    src, pos = s, 0; nxt(); p = expression()
    if sym != chr(0): raise Exception("unexpected character at " + str(pos))
    return p

The task is write convert infix expressions according to the above grammar to prefix expressions. Test cases are below. Copy the above grammar to the cell below and add attribute rules to `expression`, `term`, and `factor` for a synthesized attribute that returns the prefix form of the parsed expression:

    expression(e) → ws term(e) { '+' ws term(f) « e := '+' 'e' 'f' » }
    term(e) → factor(e) { '*' ws factor(f) « e := '*' 'e' 'f' » }
    factor(e) → (integer(e) | '(' expression(e) ')' ) ws
    integer(p) → digit(p) { digit(q) « p := q + p» } ws
    digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    ws → { ' ' }

Now implement your attribute grammar. Copy and paste the above implementation into the cell below and modify it appropriate!

In [22]:
pos: int; sym: str; src: str

def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos + 1
    else: sym = chr(0) # end of input symbol

def expression() -> str:
    # expression → term { '+' ws term }
    ws(); e = term()
    while sym == '+': nxt(); ws(); e = '+' + ' ' + e + ' ' + term() 
    return e

def term() -> str:
    # term → factor { '*' ws factor }
    p = factor()
    while sym == '*': nxt(); ws(); p = '*' + ' ' + p + ' ' + factor()
    return p

def factor() -> str:
    # factor → (integer | '(' expression ')' ) ws
    if '0' <= sym <= '9': return integer()
    elif sym == '(':
        nxt(); e = expression()
        if sym == ')': nxt(); ws(); return e
        else: raise Exception("')' expected at " + str(pos))
    else: raise Exception("invalid character at " + str(pos))

def integer() -> str:
    # integer(p) → digit(p) { digit(q) « p := q + p» } ws
    p = digit()
    while '0' <= sym <= '9': p += digit()
    ws()
    return p

def digit() -> str:
    # digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    # '0' <= sym <= '9'
    p = sym; nxt()
    return p

def ws():
    # ws → { ' ' }
    while sym == ' ': nxt()

def convert(s) -> str:
    global src, pos;
    src, pos = s, 0; nxt(); p = expression()
    if sym != chr(0): raise Exception("unexpected character at " + str(pos))
    return p


Here are some test cases. Your implementation will be tested with further test cases.

In [23]:
assert convert('1 + 2') == '+ 1 2'
assert convert('((1) + (2))') == '+ 1 2'
assert convert('1 + 2 + 3') == '+ + 1 2 3'
assert convert('(1 + 2) + 3') == '+ + 1 2 3'
assert convert('1 + (2 + 3)') == '+ 1 + 2 3'

In [24]:
assert convert('1 * 2') == '* 1 2'
assert convert('((1) * (2))') == '* 1 2'
assert convert('1 * 2 * 3') == '* * 1 2 3'
assert convert('(1 * 2) * 3') == '* * 1 2 3'
assert convert('1 * (2 * 3)') == '* 1 * 2 3'

In [25]:
assert convert('1 + (2 + 3)') == '+ 1 + 2 3'
assert convert('1 + 2 * 3') == '+ 1 * 2 3'
assert convert('1 * 2 + 3') == '+ * 1 2 3'
assert convert('(1 + 2) * 3') == '* + 1 2 3'

Suppose you want to convert prefix expressions to postfix expressions. First, give a grammar for prefix expressions. The grammar must allow all the prefix expressions from the above test cases; you do not need to be concerned about white space (`ws`), and you do not need to repeat the productions for `integer` and `digit` if they are unchanged. The grammar must satisfy the conditions for recursive descent parsing.

    expression → { '+' term term }
    term → { '*' factor factor }
    factor → (integer | '(' expression ')' ) ws
    integer(p) → digit(p) { digit(q) « p := q + p» } ws
    digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    ws → { ' ' }

Now add attribute rules such that the synthesized attribute of the start symbol is the conversion of the prefix expression to postfix.

    expression(e) → { '+' term(e) term(f) « p := 'e' 'f' '+' »}
    term → { '*' factor factor }
    factor → (integer | '(' expression ')' ) ws
    integer(p) → digit(p) { digit(q) « p := q + p» } ws
    digit(p) → '0' « p := '0' » | ... | '9' « p := '9' »
    ws → { ' ' }